# Analysis

## Logical Analysis

In [ ]:
import math

capital_letters = 26
small_letters = 26
digits = 10
special_characters = 33
all_characters = 95
l = 13
passphrase = (math.comb(l,4)) * capital_letters * small_letters * digits * special_characters * all_characters ** (l-4)

mnemonic_BIP39 = 2048
unique_biometric = 8000000000

possible_combinations = passphrase * mnemonic_BIP39 * unique_biometric
print(possible_combinations)
print(2**128)

1647020726677939641600000000000000000000
340282366920938463463374607431768211456
